In [ ]:
pip install torch

In [ ]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install opencv-python

     ---------------------------------------- 35.6/35.6 MB 8.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#  TCGA dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torchvision
from torchvision import models, datasets
from torchvision.transforms import transforms
from torch.utils.data import WeightedRandomSampler, DataLoader, SubsetRandomSampler, Subset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import random
import matplotlib.pyplot as plt
# from torchsummary import summary
import numpy as np
import cv2 as cv
torch.manual_seed(0)
torch.cuda.manual_seed(0)
# %matplotlib inline
random.seed(0)

In [ ]:
batch_size = 1
input_size = 64
num_epochs = 47
learning_rate = 0.001
warm_restart = 12
weight_decay = 0.00001

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.fc0 = nn.Linear(input_dim, input_dim//4)
        self.fc1 = nn.Linear(input_dim//4, input_dim//16)
        self.batchnorm1 = nn.BatchNorm1d(input_dim//16)
        self.fc2 = nn.Linear(input_dim//16, input_dim//32)
        self.fc3 = nn.Linear(input_dim//32, input_dim//128)
        self.batchnorm2 = nn.BatchNorm1d(input_dim//128)
        self.fc4 = nn.Linear(input_dim//128, input_dim//32)
        self.fc5 = nn.Linear(input_dim//32, input_dim//16)
        self.batchnorm3 = nn.BatchNorm1d(input_dim//16)
        self.fc6 = nn.Linear(input_dim//16, input_dim//4)
        self.fc7 = nn.Linear(input_dim//4, input_dim)
        self.relu = nn.ReLU()
  
    def forward(self, x):
        x = self.relu(self.fc0(x))
        x = self.relu(self.batchnorm1(self.fc1(x)))
        x = self.relu(self.fc2(x))
        x = self.relu(self.batchnorm2(self.fc3(x)))
        x = self.relu(self.fc4(x))
        x = self.relu(self.batchnorm3(self.fc5(x)))
        x = self.relu(self.fc6(x))
        x = self.fc7(x)
        return(x)

In [ ]:
model = Model(input_size*input_size, input_size*input_size)

In [ ]:
model.to(device)

Model(
  (fc0): Linear(in_features=4096, out_features=1024, bias=True)
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (batchnorm1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=32, bias=True)
  (batchnorm2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=32, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=256, bias=True)
  (batchnorm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc6): Linear(in_features=256, out_features=1024, bias=True)
  (fc7): Linear(in_features=1024, out_features=4096, bias=True)
  (relu): ReLU()
)

In [ ]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 72B3-5164

 Directory of C:\Users\Asus\Documents

17/09/2022  11:50 AM    <DIR>          .
17/09/2022  11:50 AM    <DIR>          ..
17/09/2022  11:23 AM    <DIR>          .ipynb_checkpoints
17/09/2022  11:25 AM    <DIR>          .spyproject
17/09/2022  11:50 AM        26,778,802 ML mini projedct.ipynb
17/09/2022  11:27 AM             2,329 ML mini projedct.py
17/09/2022  11:09 AM        35,991,944 model5.pth
17/09/2022  11:36 AM        35,991,944 model7.pth
29/06/2020  05:53 PM            46,255 passport_size_photo.jpg
23/05/2021  04:37 PM           184,365 There is No.mp3
23/05/2021  05:31 PM           184,365 ThereisNo.mp3
23/05/2021  05:51 PM            14,934 Untitled.ipynb
               8 File(s)     99,194,938 bytes
               4 Dir(s)  95,608,389,632 bytes free


In [ ]:
model_path = "model7.pth"
# ‪model.load_state_dict(model_path)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
def count_parameters(Model): 
  return sum(p.numel() for p in Model.parameters() if p.requires_grad), sum(p.numel() for p in Model.parameters())
print(count_parameters(model))

(8994656, 8994656)


In [ ]:
criterion = nn.MSELoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
schedular = CosineAnnealingWarmRestarts(optimizer,warm_restart)
criterion.to(device)

MSELoss()

In [ ]:
def get_sobel(inputs):
  sobel = [[1, 2, 1], [0, 0, 0], [-1, -2, -1]]
  sobel_kernel = torch.tensor(sobel, dtype=torch.float32).expand(inputs.size()[1], inputs.size()[1], 3, 3).to(device)
  return(sobel_kernel)
def get_previt(inputs):
  previt = [[1, 1, 1], [0, 0, 0], [-1, -1, -1]]
  previt_kernel = torch.tensor(previt, dtype=torch.float32).expand(inputs.size()[1], inputs.size()[1], 3, 3).to(device)
  return(previt_kernel)
def get_laplacian(inputs):
  laplacian = [[-1,-1,-1],[-1,8,-1],[-1,-1,-1]]
  laplacian_kernel = torch.tensor(laplacian, dtype=torch.float32).expand(inputs.size()[1], inputs.size()[1], 3, 3).to(device)
  return(laplacian_kernel)

In [ ]:
img = Image.open(r"D:\College subjects\7th sem\Machine Learning\furniture_images\furniture_images\1634011563649_New Office Cupboard 4 X 3 for sale.jpg")
convert_tensor = transforms.ToTensor()
convert_tensor(img)

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [ ]:
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog
from tkinter.filedialog import askopenfile
import cv2
import numpy as np


def upload_file():
    global img
    global w1
    f_types = [('Jpg Files', '*.jpg')]
    filename = filedialog.askopenfilename(filetypes=f_types)
    img = ImageTk.PhotoImage(file=filename)
    canvas.create_image(200, 200, anchor=tk.NW, image=img)
    w1['state'] = 'normal'

def edge_detect():
    global img1
    # code for processing
    img = cv2.imread('download.jpg')
    #canny
    img_canny = cv2.Canny(img,100,200)
    
    filename = 'savedImage.jpg'
    cv2.imwrite(filename, img_canny)
    
#     img = ImageTk.PhotoImage(file=filename)
    img = ImageTk.PhotoImage(file="savedImage.jpg")

    canvas.create_image(1000, 380, anchor=tk.SE, image=img) #need to check
    
# def prewitt():
#     global img2
#     # code for processing

#     img = cv2.imread('download.jpg')
#     if(img is not None):
#         gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_gaussian = cv2.GaussianBlur(gray,(3,3),0)

#     #canny
#     img_canny = cv2.Canny(img,100,200)

#     img2 = img_canny
#     canvas.create_image(1000, 380, anchor=tk.SE, image=img1) #need to check

    
    

top = tk.Tk()
top.title('ML Mini Project')
# Create label
l = tk.Label(top, text = "Edge Detector")
l.config(font =("Courier", 14))
l.pack()
# arranging application parameters
canvas = tk.Canvas(top, width=1200,height=550)

canvas.pack()

w=tk.Button(top, text='Upload Image', width=25, command = upload_file)
w.pack()
w1=tk.Button(top, text='Edge Map', width=25, command = edge_detect, state="disabled")
w1.pack()
w_quit=tk.Button(top, text='Exit', width=25, command = top.quit)

w_quit.pack()


# Code to add widgets will go here...
top.mainloop()